In [14]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage 
from langchain_core.messages import ToolMessage  
from langchain_core.messages import SystemMessage 
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage

In [15]:
load_dotenv()

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
@tool
def sum(a:int,b:int):
    """this is the add function"""
    return a+b

@tool
def sub(a:int,b:int):
    """this is the subtraction function"""
    return a-b 

@tool
def product(a:int,b:int):
    """this is the multiplication function"""
    return a*b  

tools=[sum,sub,product]

In [17]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.7
).bind_tools(tools)


In [ ]:
def model(state:AgentState)->AgentState:
    """this is for calling the model"""
    try:
        systemPrompt = SystemMessage(content="you are a ai agent chat bot")
        response = llm.invoke([systemPrompt] + state["messages"])
        return {"messages": [response]}
    except Exception as e:
        print(f"Error in model: {e}")
        return {"messages": []}  
                            


In [19]:

def should_continue(state: AgentState): 
    messages = state["messages"]
    last_message = messages[-1]
    # Add proper type checking
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls: 
        return "continue"
    else:
        return "end"

In [20]:
graph = StateGraph(AgentState)
graph.add_node("our_agent", model)


tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

graph.set_entry_point("our_agent")

graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue": "tools",
        "end": END,
    },
)

graph.add_edge("tools", "our_agent")

app = graph.compile()


In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"Tool calls: {message.tool_calls}")
            message.pretty_print()

inputs = {"messages": [HumanMessage(content="Add 40 + 12 and then multiply the result by 6. Also tell me a joke please.")]}
print_stream(app.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Add 40 + 12 and then multiply the result by 6. Also tell me a joke please.
Tool calls: [{'name': 'sum', 'args': {'a': 40, 'b': 12}, 'id': 'dskkws2nr', 'type': 'tool_call'}, {'name': 'product', 'args': {'a': 52, 'b': 6}, 'id': '4rengt2zh', 'type': 'tool_call'}]
================================== Ai Message ==================================
Tool Calls:
  sum (dskkws2nr)
 Call ID: dskkws2nr
  Args:
    a: 40
    b: 12
  product (4rengt2zh)
 Call ID: 4rengt2zh
  Args:
    a: 52
    b: 6
================================= Tool Message =================================
Name: product

312
================================== Ai Message ==================================

Why couldn't the bicycle stand up by itself? Because it was two-tired.
